In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hemul
hemul.USE_FPGA=False
from hemul import loader
he = loader.load()
from hemul import heaan
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv
from muxcnn.utils import get_channel_last, get_conv_params
from muxcnn.hecnn_par import MultParPack
from muxcnn.hecnn_par import forward_convbn_par

Using CPU version HEAAN
Using CPU version HEAAN
Using CPU version HEAAN
Using CPU version HEAAN


## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

img_tensor = load_img("./cute.jpg", hi=32, wi=32)

## Image

In [4]:
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1 # initial ki
hi, wi, ch = imgl.shape

# FHE

In [5]:
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

# try 40 & 800
hec = heaan.HEAANContext(15, 40, 800, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
HEAAN CKKS setup is ready 


In [6]:
fhemodel = ResNetHEAAN(model, hec)

ctxt = fhemodel.pack_img_ctxt(img_tensor)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


# forward early

In [7]:
ctxt0, outs0 = fhemodel.forward_early(ctxt, 1, 32, 32)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
ct_a 40 800 [-7.49019623e-01 -9.13725497e-01 -5.52941202e-01 -8.43137264e-01
  1.18700309e-09 -4.11764677e-01  9.80392702e-02 -8.50980402e-01
 -2.75902376e-09 -2.47058813e-01  1.29411815e-01 -6.78431389e-01
 -9.98199988e-10  8.23529926e-02  2.94117688e-01  2.31372596e-01
  6.34764651e-09  1.92156914e-01  1.52941235e-01  1.29411821e-01
  6.57313531e-10  2.78431417e-01 -5.52941206e-01 -5.45098070e-01
 -1.21937052e-09  1.05882405e-01  4.98039241e-01  4.50980423e-01
 -8.91664277e-11  7.41176484e-01  7.41176488e-01 -4.27450954e-01
  2.32312150e-09]
ct_d 40 720
40 760
40 760
Check 1
[ 1.23083786  0.48697917  0.27534379 ... -0.65056945 -0.61289198
 -0.47934081]
APPR 40 720
[ 1.23083786  0.48697917  0.27534379 ... -0.65056945 -0.61289198
 -0.47934081]
min max -5.745771387159031 5.543758366855267
APPR 40 520
[ 0.60359733  0.25016226  0.14227704 ... -0.33194524 -0.3

In [8]:
dec = hec.decrypt(ctxt0)
print(dec[::1000])

Using CPU version HEAAN
[ 1.23083790e+00 -3.27018611e-09  9.00872387e-01  4.33703741e-10
  1.03879863e+00 -1.65501832e-03  1.72884925e-02  2.26538683e-01
  7.03368487e-09 -4.52667923e-10 -3.11528386e-10 -6.20041315e-09
 -1.51824624e-10  4.46616290e-01  4.32714040e-10  1.06025075e+00
 -2.02556254e-08  5.96188892e-10  2.69868565e-01  3.97408023e+00
  1.03376648e+00  7.15351707e-09  5.28608069e-01  2.52203241e+00
 -2.12115824e-09  8.40949255e-02 -4.77262249e-08 -2.99083129e-08
  7.20822758e-09  6.15333972e-01  5.37446547e-01  4.76048093e-01
 -6.87114449e-09]


In [9]:
hec.rescale(ctxt0)

In [10]:
ctxt1, outs1 = fhemodel.forward_bb(model.layer1[0], ctxt0, outs0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
ct_a 40 520 [ 1.23083790e+00 -3.29777273e-09  9.00872387e-01  3.86034852e-10
  1.03879862e+00 -1.65501896e-03  1.72884923e-02  2.26538683e-01
  6.65668712e-09 -9.50798181e-10  5.78189352e-10 -6.42239504e-09
 -1.79728203e-10  4.46616291e-01  5.22594154e-10  1.06025075e+00
 -2.04062856e-08  6.16874283e-10  2.69868565e-01  3.97408023e+00
  1.03376648e+00  7.85200960e-09  5.28608069e-01  2.52203241e+00
 -2.05317947e-09  8.40949257e-02 -4.73317680e-08 -2.94794796e-08
  7.40494243e-09  6.15333972e-01  5.37446547e-01  4.76048092e-01
 -6.59084803e-09]
ct_d 40 440
40 480
40 480
40 480
40 480
40 480
40 480
40 480
40 480
APPR 40 440
[ 0.86425901  1.07560649  0.71689009 ...  0.77778073  1.14699243
 -0.12684554]
min max -7.502170828275916 4.664819436963656
APPR 40 240
[ 0.43581931  0.53437551  0.36458051 ...  0.39424049  0.56656664
 -0.0656868 ]
min max -1.177562744899

In [11]:
ctxt2, outs2 = fhemodel.forward_bb(model.layer2[0], ctxt1, outs1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
ct_a -160 360 [-7.55495086e+157 -5.19874646e+156 -1.43452015e+158  5.92495583e+157
  3.14469788e+158  7.73574841e+156  1.88512267e+158  3.21361038e+157
  5.50819414e+157 -3.15773732e+158 -2.71470983e+158 -1.31103105e+158
  7.90035679e+157  6.77592755e+157  6.99490466e+157  1.68226352e+158
  2.11461367e+158 -1.94913619e+157  2.39647162e+158 -1.19243521e+158
  1.16999473e+158  8.14904659e+157 -1.97164266e+157  1.67444649e+158
  1.86504860e+158  1.89382521e+158 -6.20627674e+157  2.36085322e+158
 -4.67296013e+156  1.73012432e+158  2.73635047e+157 -1.07806796e+158
 -3.90507233e+157]
ct_d 40 280
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
APPR 40 280
[0.97729385 0.97729385 0.97729385 ... 0.97729385 0.97729385 0.97729385]
min max 0.9772938467112908 0.9772938467112908
APPR 40 80
[0.48910557 0.489


KeyboardInterrupt



In [ ]:
ctxt3, outs3 = fhemodel.forward_bb(model.layer3[0], ctxt2, outs2)

In [5]:
# early conv and bn
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins0)

In [6]:
# Muxed
U, ins1, outs1 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins1)

out1, un1 = forward_convbn_par(model.conv1, 
                              model.bn1, ct_a, ins1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.7468 s


## Activation